Praktikum NLP:

ToDo:
- Load dataset √

- Dependency Parsing

- Extracting aspects (phrases/sentiment-words)

    - Summarize aspects for each hotel

    - Calculate sentiment scores for each aspect (TextBlob, VADER, etc...)

- Analyse 5 most common aspects

- Dump into graphs for the presentation

In [3]:
#Load data

import pandas as pd
import re
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import string

with open('Hotel_Reviews_part1.csv') as file:
    df = pd.read_csv(file)




In [4]:
## Clean data

# Delete redundant and empty reviews

def preprocessing_reviews(df):
    df.drop(df[ df['Negative_Review'] == 'No Negative'].index)
    df.drop(df[ df['Positive_Review'] == 'No Positive'].index)
    df.drop_duplicates(keep=False)

# Split into sentences and add dots to positive reviews and negative reviews
    for text in df['Positive_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t|n\'t|nt', ' not',complete_review)
        df['Positive_Review'] = df['Positive_Review'].replace([text], complete_review)

    for text in df['Negative_Review']:
        parts_of_review = re.findall('[A-Z][^A-Z]*', text)
        for index, sentence in enumerate(parts_of_review):
            if "I " or "i " in sentence:
                parts_of_review[index : index +1] = [''.join(parts_of_review[index : index + 1])]
            parts_of_review[index] = sentence.strip() + "."
        complete_review = ' '.join(parts_of_review)

        # correct 'n t'|'nt'|'n t' to ' not'
        complete_review = re.sub(r'n\s+t|n\'t|nt', ' not',complete_review)
        df['Negative_Review'] = df['Negative_Review'].replace([text], complete_review)

    return df



df = preprocessing_reviews(df)
pd.set_option("display.max_colwidth", -1)

# Quick check if wanted
#print(df['Negative_Review'])
#print(df['Positive_Review'])




# Remove stopwords

#nlp = spacy.load("en_core_web_sm")
#stopword_list = nltk.corpus.stopwords.words('english')


# lower case


# Lemmatization






/var/folders/xw/_l0rmmc5059811kqz4qt5k4c0000gn/T/ipykernel_97473/492218809.py:40: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [5]:
# Dependency parsing

import nltk



def dependency_parsing(df):
    for sentence in df:
        break
  #      doc = nlp(sentence)




In [6]:
# In Addition:

# make Sentiment Analysis (only simple one: Textblob) for positive and negative reviews and check, if it fits,
#                otherwise drop the review
# -> example: "I am so angry that i made this post available via all possible sites i use when planing my trips so
#              no one will make the mistake of booking this place"
# maybe handle orthographical mistakes before splitting into sentences
#



In [7]:
# Define rules for extracting aspects: keywords:

# Aspect food: restaurant, breakfast, dinner, lunch, bar
# Aspect helpfulness: staff, personell, clerk, service, check-in
# Aspect price: cheap, expensive, euros, ...?
# Aspect cleanliness: bed, matress, bathroom, floor, sheets, "changed daily", shower
# Aspect features/equipment:  USB-port, wifi, airconditioning, showergel, shampoo, furniture, roomservice, bathtub
# Aspect connectivity: bus, tram, train, airportshuttle, parking
# Aspect property: stairs (steep), building, (under-)construction, renovation, location, citycenter, sightseeing


Presentation: Spatial-Mapping where hotels are // where guests are from
=> maybe what aspects are mentioned by primarily by guests by nationality or positive/negative in general

